In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv
/kaggle/input/rainfall-prediction-using-machine-learning/Rainfall.csv


In [2]:
seed = 1

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s5e3/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e3/test.csv')
train_org = pd.read_csv('/kaggle/input/rainfall-prediction-using-machine-learning/Rainfall.csv')
# target = train['rainfall']

In [4]:
train.columns

Index(['id', 'day', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection',
       'windspeed', 'rainfall'],
      dtype='object')

In [5]:
train.drop(columns='id', inplace=True)

In [6]:
train_org.columns

Index(['day', 'pressure ', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'humidity ', 'cloud ', 'rainfall', 'sunshine', '         winddirection',
       'windspeed'],
      dtype='object')

In [7]:
train.columns = train.columns.str.strip().str.lower()
train_org.columns = train_org.columns.str.strip().str.lower()

In [8]:
# train.drop(columns=['id'], inplace=True)

In [9]:
common_cols = ['day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 
               'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 
               'windspeed', 'rainfall']
train_org[common_cols].dtypes

day                int64
pressure         float64
maxtemp          float64
temparature      float64
mintemp          float64
dewpoint         float64
humidity           int64
cloud              int64
sunshine         float64
winddirection    float64
windspeed        float64
rainfall          object
dtype: object

In [10]:
train_org['rainfall']

0      yes
1      yes
2      yes
3      yes
4      yes
      ... 
361    yes
362    yes
363    yes
364    yes
365     no
Name: rainfall, Length: 366, dtype: object

In [11]:
map_rainfall = {
    'yes':1,
    'no':0
}
train_org['rainfall'] = train_org['rainfall'].map(map_rainfall)

In [12]:
for col in common_cols :
    if col!='rainfall':
        train[col] = train[col].astype('float64')
        train_org[col] = train_org[col].astype('float64')

In [13]:
train_comp = pd.concat([train, train_org], axis=0, ignore_index=True)

In [14]:
train_comp['id'] = np.arange(0, train_comp.shape[0])

In [15]:
train_comp['group_year'] = train_comp['id'] // 365

In [16]:
train_comp.isnull().sum()

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    1
windspeed        1
rainfall         0
id               0
group_year       0
dtype: int64

In [17]:
# import matplotlib.pyplot as plt
# plt.pie(train['rainfall'].value_counts(), autopct='%.0f%%', labels=['positive', 'negative'])

In [18]:
train.isnull().sum()

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    0
windspeed        0
rainfall         0
dtype: int64

In [19]:
train.dtypes

day              float64
pressure         float64
maxtemp          float64
temparature      float64
mintemp          float64
dewpoint         float64
humidity         float64
cloud            float64
sunshine         float64
winddirection    float64
windspeed        float64
rainfall           int64
dtype: object

In [20]:
train.nunique()

day              365
pressure         236
maxtemp          219
temparature      198
mintemp          199
dewpoint         218
humidity          49
cloud             78
sunshine         120
winddirection     35
windspeed        223
rainfall           2
dtype: int64

In [21]:
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score


group_years = train_comp['group_year']
def cv_score(X, y, model, n_splits=5, seed=1):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    # gkf = GroupKFold(n_splits=n_splits)
    scores = []
    for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        model.fit(X_train.values, y_train.values)
        preds = model.predict_proba(X_valid.values)[:,1]
        score = roc_auc_score(y_valid, preds)
        scores.append(score)
        print(f'score at fold {i+1} :', score)
    print(f'mean score for all folds :', np.mean(scores))

In [22]:
train_comp.columns

Index(['day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed',
       'rainfall', 'id', 'group_year'],
      dtype='object')

In [23]:
def artificial_features(df):
    # df['temp_diff'] = df['maxtemp'] - df['mintemp']
    df['day'] = pd.to_datetime(df['day'], errors='coerce')

    df['month'] = df['day'].dt.month
    # Monsoon/rainy season indicator (adjust months based on your region)
    df['is_monsoon'] = df['month'].isin([6,7,8]).astype(int)  # Example for Northern Hemisphere summer monsoon
    # df['day_of_week'] = df['day'].dt.dayofweek
    # df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    df['avg_temp'] = (df['maxtemp'] + df['mintemp']) / 2
    # df['temp_deviation'] = df['temparature'] - df['avg_temp']
    df['dew_point_depression'] = df['temparature'] - df['dewpoint']
    df['temp_diff_from_avg'] = df['temparature'] - df['avg_temp']  # Deviation from average temp
    # df['temp_range'] = df['maxtemp'] - df['mintemp']  # Temperature range
    # # Wind direction - sine and cosine transformation
    df['wind_dir_rad'] = np.deg2rad(df['winddirection'])
    # df['wind_dir_sin'] = np.sin(df['wind_dir_rad'])
    df['wind_dir_cos'] = np.cos(df['wind_dir_rad'])
    # df.drop(columns=['wind_dir_rad'], inplace=True)
     # Temperature deviation: current avg_temp minus rolling average
    # # Wind chill factor (simplified version)
    # df['wind_chill'] = 13.12 + 0.6215 * df['temparature'] - 11.37 * (df['windspeed']**0.16) + 0.3965 * df['temparature'] * (df['windspeed']**0.16)
    # Air density proxy (simplified)
    # df['air_density'] = df['pressure'] / (df['temparature'] + 273.15)  # Using ideal gas law (P/(RT))

    # Saturation vapor pressure (Teten's formula)
    # df['es'] = 0.6108 * np.exp(17.27 * df['temparature'] / (df['temparature'] + 237.3))  # in kPa
    # df['actual_vapor_pressure'] = 0.6108 * np.exp(17.27 * df['dewpoint'] / (df['dewpoint'] + 237.3))
    # df['vapor_pressure_deficit'] = df['es'] - df['actual_vapor_pressure']  # Drives evaporation
    # # Interaction features
    # df['humidity_temp'] = df['humidity'] * df['temparature']
    # df['cloud_sunshine'] = df['cloud'] * df['sunshine']
    
    # # Rolling statistical features
    # df['rolling_temp_mean'] = df['avg_temp'].rolling(window=7).mean()
    # df['rolling_wind_mean'] = df['windspeed'].rolling(window=7).mean()
    df['rolling_temp_mean'] = df['avg_temp'].rolling(window=7, min_periods=1).mean()
    df['rolling_wind_mean'] = df['windspeed'].rolling(window=7, min_periods=1).mean()
    # Wind vector components (assuming winddirection=0° is North)
    # df['wind_u'] = df['windspeed'] * np.sin(df['wind_dir_rad'])  # East-West component
    # df['wind_v'] = df['windspeed'] * np.cos(df['wind_dir_rad'])  # North-South component

    # # Wind persistence (e.g., consistent easterly winds)
    # df['wind_speed_change'] = df['windspeed'].diff(1)
    # Lifted Index approximation (requires vertical temp profile; simplified)
    # df['lifted_index'] = df['mintemp'] - df['dewpoint']  # Lower values → more instability

    # Simplified K-index (thunderstorm potential)
    df['k_index'] = (df['maxtemp'] - df['mintemp']) + df['dewpoint'] - (df['temparature'] - df['dewpoint'])
    
    # Pressure anomaly vs rolling mean (identifies sudden drops)
    # df['pressure_anomaly'] = df['pressure'] - df['pressure'].rolling(3, min_periods=1).mean()
    # df['k_humidity_interaction'] = df['k_index'] * df['humidity']
    # df['total_totals'] = (df['maxtemp'] - df['mintemp']) + (df['dewpoint'] - 50)
    # df['k_index_lag1'] = df['k_index'].shift(1)
    # df['k_index_lag2'] = df['k_index'].shift(2)
    # Exponential moving averages (recent trends matter more)
    # df['ema_temp_3'] = df['avg_temp'].ewm(span=3, adjust=False).mean()
    # df['ema_humidity_5'] = df['humidity'].ewm(span=5, adjust=False).mean()

    # Rolling pressure variability (storminess indicator)
    # df['rolling_pressure_std_5'] = df['pressure'].rolling(5, min_periods=1).std()
    # Precipitable water proxy (column moisture)
    # df['precipitable_water'] = df['humidity'] * df['dew_point_depression']
     # Interaction and ratio features
    # df['humidity_temp_ratio'] = df['humidity'] / (df['avg_temp'] + 1e-6)  # Relationship between moisture and temperature
    # df['sunshine_cloud_ratio'] = df['sunshine'] / (df['cloud'] + 1e-6)    # How clear or overcast the day is
    # Additional rolling statistic: 7-day rolling variance for temperature
    # df['rolling_temp_var'] = df['avg_temp'].rolling(window=7, min_periods=1).var()
    # df['rolling_humidity_mean'] = df['humidity'].rolling(window=7).mean()
    df['temp_deviation'] = df['avg_temp'] - df['rolling_temp_mean']
    # df['humidity_sq'] = df['humidity'] ** 2
    # df['avg_temp_sq'] = df['avg_temp'] ** 2
    # Pressure trend: 3-day rolling mean and the deviation from it
    # df['rolling_pressure_mean_3'] = df['pressure'].rolling(window=3, min_periods=1).mean()
    # df['pressure_trend'] = df['pressure'] - df['rolling_pressure_mean_3']
    # df.drop(columns=['rolling_pressure_mean_3'], inplace=True)
    # # Lag features
    # df['temp_lag_1'] = df['avg_temp'].shift(1)
    # df['humidity_lag_1'] = df['humidity'].shift(1)
    # df['windspeed_lag_1'] = df['windspeed'].shift(1)
    
    # # Pressure-Temperature interaction
    # df['pressure_temp_interaction'] = df['pressure'] * df['avg_temp']
    # # Wind-Speed-Temperature interaction
    # df['windspeed_temp_interaction'] = df['windspeed'] * df['avg_temp']
    
    # # Season feature
    # df['season'] = df['month'].apply(lambda x: 'Spring' if 3 <= x <= 5 else
    #                                   'Summer' if 6 <= x <= 8 else
    #                                   'Autumn' if 9 <= x <= 11 else 'Winter')

    # for c in ['pressure', 'maxtemp', 'temparature', 'humidity']:
    for c in ['humidity', 'temparature', 'pressure', 'maxtemp']:
        for gap in [1]:
            df[c+f"_shift{gap}"] = df[c].shift(gap)
            df[c+f"_diff{gap}"] = df[c].diff(gap)
    # for c in ['pressure', 'maxtemp', 'temparature', 'mintemp','dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection','windspeed']:
    #     for gap in [1]:
    #         df[c+f"_shift{gap}"]=df[c].shift(gap)
    #         df[c+f"_diff{gap}"]=df[c].diff(gap)

    # # Binary encoding for season
    # df = pd.get_dummies(df, columns=['season'], drop_first=True)
    # # Drop original 'day' column
    

    # df['hum_sunshine_int'] = df['humidity'] * df['sunshine']
    # df['hum_sunshine_ratio'] = df['humidity'] / (df['sunshine']+1e-6)
    # df['cloud_sunshine_int'] = df['cloud'] * df['sunshine']
    # df['cloud_sunshine_ratio'] = df['cloud'] / (df['sunshine']+1e-6)
    # df['cloud_pressure_int'] = df['cloud'] * df['pressure']
    # df['cloud_pressure_ratio'] = df['cloud'] / (df['pressure']+1e-6)
    # base_features = [
    #     'day', 'pressure', 'maxtemp', 'temparature', 'mintemp',
    #     'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed'
    # ]
    
    # # Verify all base features exist
    # missing = [f for f in base_features if f not in df.columns]
    # if missing:
    #     raise ValueError(f"Missing base features: {missing}")

    # # List of all 55 interaction pairs 
    # df['day'] = df['day'].astype(int)
    # interaction_pairs = [
    #     ('day', 'pressure'), ('day', 'maxtemp'), ('day', 'temparature'), 
    #     ('day', 'mintemp'), ('day', 'dewpoint'), ('day', 'humidity'),
    #     ('day', 'cloud'), ('day', 'sunshine'), ('day', 'winddirection'),
    #     ('day', 'windspeed'), ('pressure', 'maxtemp'), ('pressure', 'temparature'),
    #     ('pressure', 'mintemp'), ('pressure', 'dewpoint'), ('pressure', 'humidity'),
    #     ('pressure', 'cloud'), ('pressure', 'sunshine'), ('pressure', 'winddirection'),
    #     ('pressure', 'windspeed'), ('maxtemp', 'temparature'), ('maxtemp', 'mintemp'),
    #     ('maxtemp', 'dewpoint'), ('maxtemp', 'humidity'), ('maxtemp', 'cloud'),
    #     ('maxtemp', 'sunshine'), ('maxtemp', 'winddirection'), ('maxtemp', 'windspeed'),
    #     ('temparature', 'mintemp'), ('temparature', 'dewpoint'), ('temparature', 'humidity'),
    #     ('temparature', 'cloud'), ('temparature', 'sunshine'), ('temparature', 'winddirection'),
    #     ('temparature', 'windspeed'), ('mintemp', 'dewpoint'), ('mintemp', 'humidity'),
    #     ('mintemp', 'cloud'), ('mintemp', 'sunshine'), ('mintemp', 'winddirection'),
    #     ('mintemp', 'windspeed'), ('dewpoint', 'humidity'), ('dewpoint', 'cloud'),
    #     ('dewpoint', 'sunshine'), ('dewpoint', 'winddirection'), ('dewpoint', 'windspeed'),
    #     ('humidity', 'cloud'), ('humidity', 'sunshine'), ('humidity', 'winddirection'),
    #     ('humidity', 'windspeed'), ('cloud', 'sunshine'), ('cloud', 'winddirection'),
    #     ('cloud', 'windspeed'), ('sunshine', 'winddirection'), ('sunshine', 'windspeed'),
    #     ('winddirection', 'windspeed')
    # ]

    # # Create interaction features
    # for (f1, f2) in interaction_pairs:
    #     df[f'{f1}_{f2}'] = df[f1] * df[f2]
    df.drop(columns=['day', 'month', 'is_monsoon'], inplace=True)
    return df
X = train_comp.drop(columns=['rainfall', 'id', 'group_year'])
y = train_comp['rainfall']
X = X.fillna(0)
test = test.fillna(0)
X = artificial_features(X)
test = artificial_features(test)
test.drop(columns='id', inplace=True)
# cols = [col for col in X.columns if col!='season']
# X['day'] = train_comp['day']
X = X.fillna(0)
test = test.fillna(0)

In [24]:
from sklearn.utils.class_weight import compute_class_weight

cw = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
cw_dict = dict(zip(np.unique(y), cw))

In [25]:
X.columns

Index(['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity',
       'cloud', 'sunshine', 'winddirection', 'windspeed', 'avg_temp',
       'dew_point_depression', 'temp_diff_from_avg', 'wind_dir_rad',
       'wind_dir_cos', 'rolling_temp_mean', 'rolling_wind_mean', 'k_index',
       'temp_deviation', 'humidity_shift1', 'humidity_diff1',
       'temparature_shift1', 'temparature_diff1', 'pressure_shift1',
       'pressure_diff1', 'maxtemp_shift1', 'maxtemp_diff1'],
      dtype='object')

In [26]:
# !pip install tabpfn

In [27]:
# !pip install cupy

In [28]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
# from tabpfn import TabPFNClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC, LinearSVC
# from cuml.svm import LinearSVC
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm
# scaler = RobustScaler()
# X_scaled = scaler.fit_transform(X)
# test_scaled = scaler.transform(test)
# X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
etc = ExtraTreesClassifier(
        n_estimators=390, max_depth=50, min_samples_split=2,
        min_samples_leaf=1, max_features=0.18, criterion='entropy',
        bootstrap=False, random_state=seed+28, n_jobs=-1
    )
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
test_scaled = scaler.transform(test)
test_scaled_df = pd.DataFrame(test_scaled, columns=test.columns)
# for c in X.columns:
#             m = X[c].mean()
#             s = X[c].std()
#             X = (X[c] - m)/s
knn = KNeighborsClassifier(n_neighbors=30)
# linear_svc = LinearSVC(C=0.1, probability=True)
# svc = SVC(kernel='linear', C=0.001,  probability=True)
rfc = RandomForestClassifier(random_state=seed, n_jobs=-1)
xgb_clf = xgb.XGBClassifier(random_state=seed, n_jobs=-1, tree_method='hist')
lr = LogisticRegression(n_jobs=-1, random_state=seed, fit_intercept=False)
xgb = xgb.XGBClassifier(random_state=seed)
cb = cb.CatBoostClassifier(random_state=seed, verbose=0)
lgb = lgbm.LGBMClassifier(random_state=seed, verbose=0)
hgb = HistGradientBoostingClassifier(random_state=seed)
ada = AdaBoostClassifier(random_state=seed)
mlp_clf = MLPClassifier(alpha=0.01, solver='adam', learning_rate_init=0.01, activation='relu', max_iter=1000, random_state=0)
# keras_clf = KerasClassifier(build_fn=create_model,epochs=100,verbose=0,random_state=0)
# tpfn = TabPFNClassifier(device='cuda',random_state=seed)
# scaler_2 = StandardScaler()
# train_samp = train.copy()
# train_scaled = scaler_2.fit_transform(train_samp.drop(columns=['rainfall', 'id', 'group_year']))
# train_scaled_df = pd.DataFrame(train_scaled)

# # test.drop(columns=['id'], inplace=True)
# test_scaled = scaler_2.transform(test)
# test_scaled_df = pd.DataFrame(test_scaled, columns=test.columns)

target_main = train['rainfall']
cv_score(X, y, etc, n_splits=5)

score at fold 1 : 0.8975677830940989
score at fold 2 : 0.8759039775010043
score at fold 3 : 0.9070309361189234
score at fold 4 : 0.8858577742065087
score at fold 5 : 0.9239725753577995
mean score for all folds : 0.898066609255667


0.8882183984200667(no_artificial) vs 0.8868988357279266(all_artificial) vs 0.8871090452504493(cloud removed)
vs 0.8879963509707365(temp_only)

In [29]:
# print(4//365)

In [30]:
# samp_train = train.fillna(0)
# samp_y = train['rainfall']
# cv_score(samp_train.drop(columns='rainfall'), samp_y, etc, n_splits=5)

In [31]:
# import numpy as np
# from sklearn.model_selection import StratifiedKFold
# from scipy.stats import rankdata
# from sklearn.metrics import roc_auc_score
# from sklearn.base import clone

# # Define base models - ExtraTrees variations
# base_models = [
#     ExtraTreesClassifier(n_estimators=200, max_depth=55, random_state=seed, n_jobs=-1),
#     ExtraTreesClassifier(n_estimators=300, max_depth=None, min_samples_split=4, 
#                         min_samples_leaf=2, max_features=0.3, bootstrap=False, 
#                         criterion='gini', random_state=seed+6, n_jobs=-1),
#     ExtraTreesClassifier(n_estimators=400, max_depth=40, min_samples_split=2, 
#                         min_samples_leaf=1, max_features=0.4, bootstrap=False, 
#                         criterion='gini', random_state=seed+7, n_jobs=-1),
#     ExtraTreesClassifier(
#         n_estimators=390, max_depth=50, min_samples_split=2,
#         min_samples_leaf=1, max_features=0.18, criterion='entropy',
#         bootstrap=False, random_state=seed+28, n_jobs=-1
#     )
# ]

# def rank_ensemble_cv(X, y, base_models, n_splits=5, val_size=0.2, random_state=seed):
#     """
#     Simplified rank-based ensemble validation with weight optimization
#     Fixes included for proper DataFrame indexing
#     """
#     # Initialize stratified k-fold cross-validator
#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
#     # Arrays to store results
#     final_preds = np.zeros(len(y))  # Stores final blended predictions
#     weight_history = []  # Records weights used in each fold

#     # Outer cross-validation loop
#     for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):

#         X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]  
#         y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]  

#         rng = np.random.RandomState(fold)
#         idx = rng.permutation(len(X_train))  # Shuffle indices
#         split = int(len(X_train) * (1 - val_size))  # 80-20 split
        
#         # Split indices into training and validation subsets
#         train_sub_idx = idx[:split]  # Training indices
#         val_sub_idx = idx[split:]    # Validation indices

#         # Get subset data using .iloc for positional indexing
#         X_tr = X_train.iloc[train_sub_idx]  # Training subset features
#         y_tr = y_train.iloc[train_sub_idx]  # Training subset targets
#         X_v = X_train.iloc[val_sub_idx]     # Validation features
#         y_v = y_train.iloc[val_sub_idx]     # Validation targets

#         # Lists to store model outputs
#         val_ranks = []  # Validation set rankings
#         test_preds = []  # Test set predictions

#         # Train and validate each base model
#         for model in base_models:
#             # Clone model to avoid contamination between folds
#             m = clone(model)
            
#             # 1. Train on training subset
#             m.fit(X_tr, y_tr)
            
#             # 2. Predict on validation subset
#             val_pred = m.predict_proba(X_v)[:, 1]
            
#             # 3. Convert predictions to ranks (0-1 scale)
#             val_ranks.append(rankdata(val_pred, method='dense'))
            
#             # 4. Retrain on full training data
#             m_full = clone(model).fit(X_train, y_train)
            
#             # 5. Predict on test fold
#             test_pred = m_full.predict_proba(X_val)[:, 1]
#             test_preds.append(test_pred)

#         # Convert validation ranks to array (samples x models)
#         val_ranks = np.array(val_ranks).T
        
#         # Optimize model weights using validation ranks
#         weights = optimize_weights(val_ranks, y_v)
#         weight_history.append(weights)
        
#         # Convert test predictions to ranks
#         test_ranks = np.array([rankdata(p, method='dense') for p in test_preds])
        
#         # Blend test predictions using optimized weights
#         blended = np.average(test_ranks, axis=0, weights=weights)
        
#         # Store blended predictions for this fold
#         final_preds[val_idx] = blended

#     return final_preds, weight_history, roc_auc_score(y, final_preds)

# def optimize_weights(val_ranks, y_val, n_trials=50):
#     """Bayesian optimization of ensemble weights with type safety"""
#     from skopt import gp_minimize

#     def objective(weights):
#         # Convert to float array first to prevent integer division
#         weights = np.array(weights, dtype=np.float64)  # FIX HERE: Force float type
        
#         # Handle zero-sum edge case (prevent division by zero)
#         weight_sum = weights.sum()
#         if np.isclose(weight_sum, 0):
#             return 1.0  # Worst possible score
            
#         weights /= weight_sum  # Safe normalization
        
#         # Calculate blended predictions using matrix multiplication
#         blended = val_ranks @ weights  # @ operator = dot product
        
#         return -roc_auc_score(y_val, blended)  # Negative for minimization

#     # Define continuous search space [0,1] for each model
#     space = [(0.0, 1.0) for _ in range(val_ranks.shape[1])]  # FIX HERE: Explicit floats
    
#     # Run Bayesian optimization with stability checks
#     res = gp_minimize(objective, space, 
#                      n_calls=n_trials, 
#                      random_state=seed,
#                      n_initial_points=10)  # Ensure proper initialization
    
#     # Normalize final weights safely
#     final_weights = np.array(res.x, dtype=np.float64)
#     weight_sum = final_weights.sum()
#     if np.isclose(weight_sum, 0):
#         return np.ones_like(final_weights) / len(final_weights)  # Fallback to equal weights
    
#     return final_weights / weight_sum

# preds, weights, auc = rank_ensemble_cv(X, y, base_models)
# print(f'Validated Ensemble AUC: {auc:.4f}')

In [32]:
# def train_final_ensemble(X_train, y_train, X_test, base_models, cv_weights):
#     """
#     Applies cross-validated weights to base models trained on full data
#     Returns blended test predictions and final model artifacts
#     """
#     # 1. Average weights across CV folds
#     avg_weights = np.mean(cv_weights, axis=0)
#     avg_weights /= avg_weights.sum()  # Ensure normalized
    
#     # 2. Train base models on full training data
#     trained_models = []
#     test_probas = []
    
#     for model in base_models:
#         # Clone to avoid contamination
#         m = clone(model).fit(X_train, y_train)
#         trained_models.append(m)
        
#         # Get test probabilities
#         probas = m.predict_proba(X_test)[:, 1]
#         test_probas.append(probas)
    
#     # 3. Convert probabilities to ranks
#     test_ranks = np.array([rankdata(p, method='dense') for p in test_probas])
    
#     # 4. Apply averaged weights
#     blended_ranks = np.average(test_ranks, axis=0, weights=avg_weights)
    
#     return {
#         'blended_preds': blended_ranks,
#         'trained_models': trained_models,
#         'final_weights': avg_weights,
#         'base_predictions': test_probas
#     }

# # Usage workflow
# # Assuming you have:
# # - X_train, y_train: Full training data
# # - X_test: New test data
# # - weights: List of weight arrays from CV

# # Get final predictions
# idx = test['id']
# test.drop(columns='id', inplace=True)
# artifacts = train_final_ensemble(X, y, test, base_models, weights)

# # Access results
# final_predictions = artifacts['blended_preds']
# model_objects = artifacts['trained_models']
# weight_vector = artifacts['final_weights']
# base_preds = artifacts['base_predictions']

# print(f"Final ensemble weights: {np.round(weight_vector, 3)}")
# print(f"Sample blended prediction: {final_predictions[0]:.4f}")

In [33]:
# from scipy.stats import rankdata

# def ranks_to_probas(blended_ranks, method='minmax'):
#     """
#     Convert blended ranks to probability-like scores (0-1 scale)
    
#     Parameters:
#     blended_ranks : array-like, shape = [n_samples]
#         Raw blended rank scores from ensemble
#     method : {'minmax', 'percentile', 'sigmoid'}
#         Conversion strategy
        
#     Returns:
#     probas : array, shape = [n_samples]
#         Converted probabilities between 0 and 1
#     """
#     if method == 'minmax':
#         # Simple linear scaling (preserves rank order)
#         min_val = blended_ranks.min()
#         max_val = blended_ranks.max()
#         return (blended_ranks - min_val) / (max_val - min_val + 1e-8)
    
#     elif method == 'percentile':
#         # True percentile ranks (accounts for distribution)
#         return rankdata(blended_ranks, method='average') / len(blended_ranks)
    
#     elif method == 'sigmoid':
#         # Sigmoid calibration (handles extreme values)
#         from scipy.special import expit
#         centered = blended_ranks - np.median(blended_ranks)
#         return expit(centered / (centered.std() + 1e-8))
    
#     else:
#         raise ValueError(f"Unknown method: {method}")

# # Usage with your existing workflow:
# final_probas = ranks_to_probas(artifacts['blended_preds'], method='percentile')

# # Sanity checks
# assert np.all(final_probas >= 0), "Negative probabilities detected"
# assert np.all(final_probas <= 1), "Probabilities exceed 1"
# print(f"Probability range: {final_probas.min():.3f}-{final_probas.max():.3f}")

In [34]:
# final_probas

In [35]:
from sklearn.base import clone
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
def generate_oof_preds(X, y, model_dict, n_splits=8, seed=1):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    oof_preds = pd.DataFrame(index=X.index)
    for name, model in model_dict.items():
        scores = []
        oof_model_preds = np.zeros(len(X))
        print(f'processing {name}')
        for i,(train_index, valid_index) in enumerate(skf.split(X, y), 1):
            clf = clone(model)
            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            clf.fit(X_train, y_train)
            preds = clf.predict_proba(X_valid)[:,1]
            score = roc_auc_score(y_valid, preds)
            oof_model_preds[valid_index] = preds
            # print(f'scores for fold {i} is :', score)
            scores.append(score)
        print('mean score across all folds', np.mean(scores))
        oof_preds[name] = oof_model_preds
    return oof_preds.sort_index()

model_dict = {
    # Top performing ETCs (score > 0.895)
    'etc_o': ExtraTreesClassifier(n_estimators=200, max_depth=55, random_state=seed, n_jobs=-1),
    # 'etc_2': ExtraTreesClassifier(n_estimators=250, max_depth=60, min_samples_split=2, 
    #                              min_samples_leaf=1, max_features=0.15, bootstrap=False, 
    #                              criterion='gini', random_state=seed+1, n_jobs=-1),
    'etc_7': ExtraTreesClassifier(n_estimators=300, max_depth=None, min_samples_split=4, 
                                 min_samples_leaf=2, max_features=0.3, bootstrap=False, 
                                 criterion='gini', random_state=seed+6, n_jobs=-1),
    'etc_8': ExtraTreesClassifier(n_estimators=400, max_depth=40, min_samples_split=2, 
                                 min_samples_leaf=1, max_features=0.4, bootstrap=False, 
                                 criterion='gini', random_state=seed+7, n_jobs=-1),
    # 'etc_11': ExtraTreesClassifier(
    #     n_estimators=250, max_depth=60, min_samples_split=3, min_samples_leaf=1,
    #     max_features=0.35, bootstrap=False, criterion='entropy',
    #     random_state=seed+11, n_jobs=-1
    # ),
    # 'etc_15': ExtraTreesClassifier(
    #     n_estimators=300, max_depth=60, min_samples_split=2, min_samples_leaf=1,
    #     max_features=0.25, bootstrap=False, criterion='entropy',
    #     random_state=seed+15, n_jobs=-1
    # ),
    # 'etc_23': ExtraTreesClassifier(n_estimators=300, max_depth=55, min_samples_split=8, 
    #                           min_samples_leaf=3, max_features=0.35, bootstrap=False, 
    #                           criterion='gini', random_state=seed+23, n_jobs=-1),
    # 'etc_22': ExtraTreesClassifier(n_estimators=250, max_depth=None, min_samples_split=2, 
    #                           min_samples_leaf=1, max_features=0.3, bootstrap=True, 
    #                           criterion='gini', random_state=seed+22, n_jobs=-1),
    'etc_28': ExtraTreesClassifier(
        n_estimators=390, max_depth=50, min_samples_split=2,
        min_samples_leaf=1, max_features=0.15, criterion='entropy',
        bootstrap=False, random_state=seed+28, n_jobs=-1
    ),
   # 'knn': KNeighborsClassifier(n_neighbors=30),
   # 'svc': SVC(kernel='poly', degree=2, coef0=0.2, probability=True)
}

#     # New Logistic Regression Variations
#     'logreg_1': LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=1000,
#                                   random_state=seed, n_jobs=-1),
#     'logreg_2': LogisticRegression(penalty='l1', C=0.8, solver='liblinear', max_iter=1000,
#                                   random_state=seed+1),
#     'logreg_3': LogisticRegression(penalty='elasticnet', C=1.2, solver='saga', l1_ratio=0.4,
#                                   max_iter=1500, random_state=seed+2),
#     'logreg_4': LogisticRegression(class_weight='balanced', C=0.6, penalty='l2',
#                                   solver='newton-cg', max_iter=1000, random_state=seed+3),
#     'logreg_5': LogisticRegression(penalty='none', solver='lbfgs', max_iter=2000,
#                                   random_state=seed+4),
#     'logreg_6': LogisticRegression(C=2.0, penalty='l2', solver='saga', max_iter=1000,
#                                   random_state=seed+5, warm_start=True),

#     # XGBoost Classifiers
#     'xgb_1': XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.1,
#                           subsample=0.8, colsample_bytree=0.8, random_state=seed),
#     'xgb_2': XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.05,
#                           subsample=0.6, colsample_bytree=0.6, gamma=0.1,
#                           random_state=seed+1),
#     'xgb_3': XGBClassifier(n_estimators=150, max_depth=3, learning_rate=0.2,
#                           subsample=1.0, colsample_bytree=1.0, reg_alpha=0.5,
#                           random_state=seed+2)
# }
# oof_preds = generate_oof_preds(X_scaled_df, y, model_dict, n_splits=5, seed=1)

In [36]:
def generate_test_preds(X_scaled_df, y, test_scaled_df, model_dict, n_splits=5, seed=1):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    preds = np.zeros(len(test_scaled_df))
    test_preds = pd.DataFrame(index=test_scaled_df.index)
    for name, model in model_dict.items():
        clf = clone(model)
        clf.fit(X_scaled_df, y)
        preds = clf.predict_proba(test_scaled_df)[:,1]
        test_preds[name] = preds

    return test_preds

# test_scaled = scaler.transform(test)
# test_scaled_df = pd.DataFrame(test_scaled, columns=test.columns)
# test_preds = generate_test_preds(X_scaled_df, y, test_scaled_df, model_dict)

In [37]:
# test_ranks += np.random.uniform(0, 1e-6, size=test_ranks.shape)

In [38]:
def evaluate_combination(weights, preds, y_true):
    ensemble_preds = np.average(preds, axis=1, weights=weights)
    return roc_auc_score(y_true, ensemble_preds)

def hill_climbing(preds, y_true, max_iter=10000, patience=9000):
    n_models = preds.shape[1]
    best_weights = np.ones(n_models) / n_models
    best_score = evaluate_combination(best_weights, preds, y_true)

    n_iter=0
    tol=0
    for i in range(max_iter):
        n_iter = i
        candidate_weights = best_weights + np.random.normal(0, 0.1, best_weights.shape)
        candidate_weights = candidate_weights / candidate_weights.sum()
        candidate_score = evaluate_combination(candidate_weights, preds, y_true)

        if candidate_score > best_score:
            best_score = candidate_score
            best_weights = candidate_weights
            tol = 0

        else:
            tol+=1
        if tol>patience:
            break

    return best_score, best_weights, n_iter

In [39]:
from scipy.stats import rankdata 
def rank_transform(column):
    return rankdata(column, method='average') / len(column)

# ranked_preds = oof_preds.apply(rank_transform).add_suffix('_rank')
# test_ranked_preds = test_preds.apply(rank_transform).add_suffix('_rank')

In [40]:
# ranked_preds += np.random.uniform(0, 1e-6, size=ranked_preds.shape)

In [41]:
# ranked_preds

In [42]:
# score, weights, n_iter = hill_climbing(ranked_preds, y)
# print(f'current highest score with hill climbing: {score}, with weights: {weights}, stopped at iter: {n_iter}')

In [43]:
# final_preds = np.average(test_ranked_preds, axis=1, weights=weights)

In [44]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import PolynomialFeatures
# logistic_meta = make_pipeline(
#     # PolynomialFeatures(degree=2, include_bias=False),
#     StandardScaler(),
#     LogisticRegression(
#         C=1,  # Equivalent to alpha=0.0001 (C = 1/alpha)
#         penalty='l2',
#         solver='lbfgs',
#         max_iter=10000,
#         class_weight='balanced',
#         random_state=seed,
#         fit_intercept=True
#     )
# )

# cv_score(ranked_preds, y, logistic_meta)

In [45]:
# from sklearn.linear_model import LogisticRegression

# lr = LogisticRegression(
#     # penalty='l1',         # Options: 'l1', 'l2', 'elasticnet', 'none'
#     # solver='saga',        # Use 'saga' for L1 or ElasticNet regularization
#     # l1_ratio=0.2,         # For ElasticNet, l1_ratio defines balance (0 = L2, 1 = L1)
#     # C=0.7,                # Inverse of regularization strength (lower = stronger regularization)
#     max_iter=10000,        # Ensure convergence for complex data
#     random_state=42       # For reproducibility
# )
# cv_score(oof_preds, y , lr)

In [46]:
from sklearn.base import clone
etc_2 = clone(etc)
etc_2.fit(X, y)

ExtraTreesClassifier(criterion='entropy', max_depth=50, max_features=0.18,
                     n_estimators=390, n_jobs=-1, random_state=29)

In [47]:
# importances = etr.feature_importances_
# importance_df = pd.DataFrame({
#     'columns': X.columns,
#     'importance': importances
# })
# importance_df = importance_df.sort_values(by='importance', ascending=False)
# importance_df

In [48]:
# idx = test['id']
# test.drop(columns='id', inplace=True)

In [49]:
preds = etc_2.predict_proba(test)[:,1]

In [50]:
sample_sub = pd.read_csv('/kaggle/input/playground-series-s5e3/sample_submission.csv')
sample_sub['rainfall'] = preds
sample_sub.to_csv('submission.csv', index=False)

In [51]:
from sklearn.feature_selection import RFECV
def optimal_features(X, y, model, n_splits=5, seed=1, scoring='roc_auc'):

    # model = ExtraTreesClassifier(n_estimators=200, max_depth=55, n_jobs=-1, random_state=seed)

    rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed), scoring=scoring)
    rfecv.fit(X, y)
    best_features = X.columns[rfecv.support_].to_list()
    best_score = max(rfecv.cv_results_['mean_test_score'])

    return best_features, best_score

In [52]:
# best_features, best_score = optimal_features(X, y, etc)

In [53]:
# best_features

In [54]:
# best_score